In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/embeds/embdeds.csv
/kaggle/input/cmc-ml-spotify-recommendations/sample_submission.csv
/kaggle/input/cmc-ml-spotify-recommendations/added_info.csv
/kaggle/input/cmc-ml-spotify-recommendations/audio_features.csv
/kaggle/input/cmc-ml-spotify-recommendations/simple_pipeline.ipynb
/kaggle/input/cmc-ml-spotify-recommendations/tracks_info.csv
/kaggle/input/cmc-ml-spotify-recommendations/train.csv
/kaggle/input/cmc-ml-spotify-recommendations/test.csv


# Читаем данные

In [2]:
train = pd.read_csv('/kaggle/input/cmc-ml-spotify-recommendations/train.csv') 
train

,index,playlist_id,track_id,exists
0,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1
1,645550,35527,1PEqh7awkpuepLBSq8ZwqD,1
2,2630414,12203,7E8pPgBY84oDaXRcqODavR,0
3,3168314,21850,0Atml4huw4Fgyk6YSHiK4M,0
4,3144464,21391,4WYDmIZrwxBHdBYdvi5oQO,0
...,...,...,...,...
2773057,2211111,4280,1CsJNkcfGXCUQhst9Wo4KL,0
2773058,3600703,29901,2VBGLp3N6JxIELDbbIhoSS,0
2773059,3879581,35199,5PEzhJIvmjFCFZcFZRze9S,0
2773060,1843050,29231,0kvWwRs634uP7K50NObw2h,1


In [3]:

track_info = pd.read_csv('/kaggle/input/cmc-ml-spotify-recommendations/tracks_info.csv')

## Удаляем константные признаки

In [4]:
nunique = track_info.nunique()
track_info = track_info[nunique[nunique != 1].index]

## Удаляем совпадающие признаки - корреляция 1 по Пирсону

In [5]:
track_info.select_dtypes(exclude='object').corr()

,track_disc_number,track_duration_ms,track_explicit,track_popularity,track_track_number,track_album_disc_number,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number
track_disc_number,1.000000,0.042756,-0.028013,-0.057675,0.037053,1.000000,0.042756,-0.028013,-0.057675,0.037053
track_duration_ms,0.042756,1.000000,-0.070079,-0.080785,-0.023023,0.042756,1.000000,-0.070079,-0.080785,-0.023023
track_explicit,-0.028013,-0.070079,1.000000,0.151694,-0.048587,-0.028013,-0.070079,1.000000,0.151694,-0.048587
track_popularity,-0.057675,-0.080785,0.151694,1.000000,-0.114244,-0.057675,-0.080785,0.151694,1.000000,-0.114244
track_track_number,0.037053,-0.023023,-0.048587,-0.114244,1.000000,0.037053,-0.023023,-0.048587,-0.114244,1.000000
track_album_disc_number,1.000000,0.042756,-0.028013,-0.057675,0.037053,1.000000,0.042756,-0.028013,-0.057675,0.037053
track_album_duration_ms,0.042756,1.000000,-0.070079,-0.080785,-0.023023,0.042756,1.000000,-0.070079,-0.080785,-0.023023
track_album_explicit,-0.028013,-0.070079,1.000000,0.151694,-0.048587,-0.028013,-0.070079,1.000000,0.151694,-0.048587
track_album_popularity,-0.057675,-0.080785,0.151694,1.000000,-0.114244,-0.057675,-0.080785,0.151694,1.000000,-0.114244
track_album_track_number,0.037053,-0.023023,-0.048587,-0.114244,1.000000,0.037053,-0.023023,-0.048587,-0.114244,1.000000


In [6]:
pip install feature-engine

Note: you may need to restart the kernel to use updated packages.


In [7]:
from feature_engine.selection import DropCorrelatedFeatures

track_info = DropCorrelatedFeatures(threshold=0.99).fit_transform(track_info)
track_info

,track_id,track_artists,track_available_markets,track_explicit,track_name,track_album_album,track_album_artists,track_album_disc_number,track_album_duration_ms,track_album_explicit,track_album_id,track_album_name,track_album_popularity,track_album_track_number
0,5SrPeOqm3SBecyiAsI9HTT,[{'external_urls': {'spotify': 'https://open.s...,[],True,Perros Blancos Freestyle,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,1.0,140288.0,True,5SrPeOqm3SBecyiAsI9HTT,Perros Blancos Freestyle,0.0,1.0
1,0YfvJaebmC1ch3vMik9yuA,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Cho Ku Rei • Que Todo el Poder del Universo Es...,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,514909.0,False,0YfvJaebmC1ch3vMik9yuA,Cho Ku Rei • Que Todo el Poder del Universo Es...,40.0,1.0
2,6bSpavIeJURyGBNMUlJydq,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Wayfaring Stranger,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,254306.0,False,6bSpavIeJURyGBNMUlJydq,Wayfaring Stranger,12.0,15.0
3,0fIBz9zLZGqrCMaAUskhk2,[{'external_urls': {'spotify': 'https://open.s...,"['AE', 'AR', 'AT', 'AU', 'BA', 'BD', 'BE', 'BH...",False,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,576653.0,False,0fIBz9zLZGqrCMaAUskhk2,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...",20.0,4.0
4,2fG23IIy7rYN8eMNbAHcS2,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Keep It To Yourself,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,196040.0,False,2fG23IIy7rYN8eMNbAHcS2,Keep It To Yourself,52.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899707,2r6pzLfa4huH3L3w7hzZXx,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Tired to Death,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,222040.0,False,2r6pzLfa4huH3L3w7hzZXx,Tired to Death,21.0,1.0
899708,0IEhU1bRVEqWCfb6Fy6w67,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Death 2 Soundcloud,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,1.0,141038.0,False,0IEhU1bRVEqWCfb6Fy6w67,Death 2 Soundcloud,39.0,1.0
899709,5WRNi2GaPQa1GBAKxKHJfS,[{'external_urls': {'spotify': 'https://open.s...,[],True,Million,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,278050.0,True,5WRNi2GaPQa1GBAKxKHJfS,Million,0.0,7.0
899710,0VwdSxxhCGc0MFvndnGw6T,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Don’t Worry About Me,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,2.0,233400.0,False,0VwdSxxhCGc0MFvndnGw6T,Don’t Worry About Me,3.0,11.0


In [8]:
import ast

#track_info['track_artists'] = track_info['track_artists'].apply(lambda x: ast.literal_eval(x)[0]['name'])
track_info['track_available_markets_len'] = track_info['track_available_markets'].apply(lambda x: len(ast.literal_eval(x))) 
track_info['release_date'] = track_info['track_album_album'].apply(lambda x: ast.literal_eval(x)['release_date'][:4]).astype('float32')
track_info['album_type'] = track_info['track_album_album'].apply(lambda x: ast.literal_eval(x)['album_type'])

In [9]:
track_info

,track_id,track_artists,track_available_markets,track_explicit,track_name,track_album_album,track_album_artists,track_album_disc_number,track_album_duration_ms,track_album_explicit,track_album_id,track_album_name,track_album_popularity,track_album_track_number,track_available_markets_len,release_date,album_type
0,5SrPeOqm3SBecyiAsI9HTT,[{'external_urls': {'spotify': 'https://open.s...,[],True,Perros Blancos Freestyle,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,1.0,140288.0,True,5SrPeOqm3SBecyiAsI9HTT,Perros Blancos Freestyle,0.0,1.0,0,2021.0,single
1,0YfvJaebmC1ch3vMik9yuA,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Cho Ku Rei • Que Todo el Poder del Universo Es...,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,514909.0,False,0YfvJaebmC1ch3vMik9yuA,Cho Ku Rei • Que Todo el Poder del Universo Es...,40.0,1.0,184,2021.0,album
2,6bSpavIeJURyGBNMUlJydq,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Wayfaring Stranger,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,254306.0,False,6bSpavIeJURyGBNMUlJydq,Wayfaring Stranger,12.0,15.0,184,2015.0,album
3,0fIBz9zLZGqrCMaAUskhk2,[{'external_urls': {'spotify': 'https://open.s...,"['AE', 'AR', 'AT', 'AU', 'BA', 'BD', 'BE', 'BH...",False,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,576653.0,False,0fIBz9zLZGqrCMaAUskhk2,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...",20.0,4.0,70,2000.0,album
4,2fG23IIy7rYN8eMNbAHcS2,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Keep It To Yourself,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,196040.0,False,2fG23IIy7rYN8eMNbAHcS2,Keep It To Yourself,52.0,9.0,184,2013.0,album
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899707,2r6pzLfa4huH3L3w7hzZXx,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Tired to Death,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,222040.0,False,2r6pzLfa4huH3L3w7hzZXx,Tired to Death,21.0,1.0,178,1991.0,album
899708,0IEhU1bRVEqWCfb6Fy6w67,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Death 2 Soundcloud,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,1.0,141038.0,False,0IEhU1bRVEqWCfb6Fy6w67,Death 2 Soundcloud,39.0,1.0,184,2018.0,single
899709,5WRNi2GaPQa1GBAKxKHJfS,[{'external_urls': {'spotify': 'https://open.s...,[],True,Million,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1.0,278050.0,True,5WRNi2GaPQa1GBAKxKHJfS,Million,0.0,7.0,0,2018.0,album
899710,0VwdSxxhCGc0MFvndnGw6T,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",False,Don’t Worry About Me,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,2.0,233400.0,False,0VwdSxxhCGc0MFvndnGw6T,Don’t Worry About Me,3.0,11.0,183,2015.0,compilation


In [10]:
track_info = track_info.drop(columns=['track_available_markets'])

## Скейлим данные, потому что будем пользоваться метрическими алгоритмами

In [11]:
from  sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
track_info[track_info.select_dtypes(exclude='object').columns] = scaler.fit_transform(track_info[track_info.select_dtypes(exclude='object').columns])

In [12]:
audio_features = pd.read_csv('/kaggle/input/cmc-ml-spotify-recommendations/audio_features.csv')

In [13]:
audio_scaler = MinMaxScaler()
audio_features[audio_features.select_dtypes(exclude='object').columns] = audio_scaler.fit_transform(audio_features[audio_features.select_dtypes(exclude='object').columns])

## Округляем время добавления трека до месяца(будем агрегировать)

In [14]:
playlist_info = pd.read_csv('/kaggle/input/cmc-ml-spotify-recommendations/added_info.csv')
playlist_info['added_at'] = pd.to_datetime(playlist_info['added_at']).dt.floor('D') +  pd.offsets.MonthBegin(-1)
playlist_info

,playlist_id,track_id,added_at,added_by_id,added_by_type
0,254,2aQpISWUBToaF84DDiTeRV,2024-02-01 00:00:00+00:00,590,user
1,17014,6EipcpKMRufBSIGLZqmmwz,2021-01-01 00:00:00+00:00,3013,user
2,18570,5IJnj6Um7jVuRPT4WGj1z6,2016-02-01 00:00:00+00:00,399,user
3,14643,3vvvpZ3ez7uCEM5uhkj1ou,2024-01-01 00:00:00+00:00,2449,user
4,12131,0P0IfSuzTC6bLUsHjmDuvW,2023-10-01 00:00:00+00:00,1014,user
...,...,...,...,...,...
1399258,19350,0W7AbEauB7cP4pidLclApe,2022-02-01 00:00:00+00:00,3042,user
1399259,14587,6Vjk8MNXpQpi0F4BefdTyq,2022-07-01 00:00:00+00:00,2449,user
1399260,27434,7795WJLVKJoAyVoOtCWqXN,2022-12-01 00:00:00+00:00,857,user
1399261,8894,5xJ0KrDk6kjRP4GjAfG3VS,2022-01-01 00:00:00+00:00,1817,user


In [15]:
playlist_info.dtypes

playlist_id                    int64
track_id                      object
added_at         datetime64[ns, UTC]
added_by_id                    int64
added_by_type                 object
dtype: object

# Обучение

In [16]:
audio_features = audio_features.drop_duplicates(subset=['id'])
audio_features[audio_features.select_dtypes(np.number).columns] = audio_features[audio_features.select_dtypes(np.number).columns].astype('float32')
playlist_info[playlist_info.select_dtypes(np.number).columns] = playlist_info[playlist_info.select_dtypes(np.number).columns].astype('float32')

In [17]:
 # df.drop(columns=['playlist_id', 'added_at'])

In [18]:
df = playlist_info.merge(track_info, how='inner',on='track_id').merge(audio_features.rename(columns={'id': 'track_id'}), how='inner', on='track_id')
df[df.select_dtypes(np.number).columns] = df[df.select_dtypes(np.number).columns].astype('float32')
df = df.drop(columns=['added_by_type','track_id'])
df = df.drop(columns=['track_artists',
                  'track_name',
                  'track_album_type',
                  'track_album_id',
                  'track_type',
                  'track_album_album',
                  'track_album_disc_number',
                  'track_album_artists',
                  'track_album_name', 'track_track'], errors='ignore')
df = df.groupby(['playlist_id', 'added_at'], as_index=False).aggregate( 
    {'added_by_id': pd.Series.nunique} | \
    {column:  lambda x: pd.Series.mode(x)[0] for column in df.drop(columns=['playlist_id', 'added_at']).select_dtypes('object').columns} \
    | {column: 'mean' for column in df.select_dtypes(np.number).drop(columns=['added_by_id']).columns
})

In [19]:
df

,added_at,added_by_id,album_type,playlist_id,track_explicit,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2017-08-01 00:00:00+00:00,1,album,0.0,0.000000,0.012873,0.000000,0.305625,0.003655,0.446332,...,0.826624,0.500000,0.094888,0.095831,0.247302,0.236988,0.605375,0.527754,0.039819,0.800000
1,2017-09-01 00:00:00+00:00,1,album,0.0,0.000000,0.011862,0.000000,0.260000,0.007147,0.123188,...,0.848168,0.666667,0.067494,0.042436,0.000133,0.077867,0.607667,0.462887,0.036677,0.800000
2,2018-02-01 00:00:00+00:00,1,album,0.0,0.000000,0.012074,0.000000,0.660000,0.007797,1.000000,...,0.825304,1.000000,0.056877,0.052811,0.000001,0.154000,0.488000,0.500282,0.037337,0.800000
3,2019-01-01 00:00:00+00:00,1,album,0.0,0.000000,0.013493,0.000000,0.410000,0.019493,0.989130,...,0.834445,0.000000,0.031437,0.099197,0.047300,0.268000,0.354000,0.512295,0.041748,0.800000
4,2020-08-01 00:00:00+00:00,1,single,0.0,0.333333,0.011556,0.333333,0.356667,0.002599,0.335145,...,0.836370,0.333333,0.055912,0.001986,0.357340,0.431000,0.803000,0.517658,0.035726,0.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149451,2022-12-01 00:00:00+00:00,1,album,36768.0,0.000000,0.015125,0.000000,0.790000,0.002339,0.642391,...,0.806812,1.000000,0.041241,0.386747,0.001636,0.130220,0.244000,0.462860,0.046822,0.680000
149452,2023-01-01 00:00:00+00:00,1,album,36768.0,0.500000,0.012534,0.500000,0.830000,0.013645,1.000000,...,0.764648,0.500000,0.051861,0.800703,0.000617,0.125000,0.208000,0.431958,0.038766,0.700000
149453,2023-02-01 00:00:00+00:00,1,album,36768.0,0.000000,0.012795,0.000000,0.825000,0.009747,1.000000,...,0.809013,0.500000,0.047725,0.034137,0.000001,0.081800,0.484000,0.632151,0.039577,0.800000
149454,2020-11-01 00:00:00+00:00,1,album,36769.0,0.000000,0.013213,0.000000,0.485000,0.001949,0.669837,...,0.827393,1.000000,0.030326,0.325803,0.000373,0.139200,0.642750,0.513147,0.040877,0.800000


In [20]:
X = train.merge(df, how='inner', on='playlist_id')
X

,index,playlist_id,track_id,exists,added_at,added_by_id,album_type,track_explicit,track_album_duration_ms,track_album_explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-07-01 00:00:00+00:00,1,single,0.117647,0.011155,0.117647,...,0.779283,0.823529,0.037813,0.585554,0.020276,0.121894,0.296641,0.389247,0.034479,0.788235
1,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-08-01 00:00:00+00:00,1,single,0.000000,0.013150,0.000000,...,0.726343,0.000000,0.030662,0.855422,0.063395,0.101150,0.428500,0.496725,0.040681,0.700000
2,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-09-01 00:00:00+00:00,1,single,0.250000,0.010305,0.250000,...,0.786116,0.500000,0.061763,0.492219,0.003583,0.216550,0.575500,0.577376,0.031835,0.800000
3,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-10-01 00:00:00+00:00,1,single,0.000000,0.009905,0.000000,...,0.820296,1.000000,0.231644,0.373494,0.000000,0.127000,0.729000,0.399782,0.030590,0.800000
4,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-11-01 00:00:00+00:00,1,single,0.000000,0.011664,0.000000,...,0.769164,1.000000,0.036867,0.706325,0.078635,0.113450,0.491000,0.414005,0.036061,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594281,1843050,29231,0kvWwRs634uP7K50NObw2h,1,2023-09-01 00:00:00+00:00,1,album,0.000000,0.013528,0.000000,...,0.749529,1.000000,0.065701,0.113589,0.914000,0.475933,0.763000,0.417247,0.041858,0.866667
14594282,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-04-01 00:00:00+00:00,1,single,0.000000,0.016165,0.000000,...,0.795941,0.533333,0.067125,0.055117,0.222907,0.130253,0.741400,0.493158,0.050057,0.793333
14594283,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-05-01 00:00:00+00:00,1,album,0.000000,0.018759,0.000000,...,0.771264,0.333333,0.055533,0.092393,0.142364,0.154644,0.825000,0.476682,0.058123,0.800000
14594284,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-06-01 00:00:00+00:00,1,album,0.000000,0.013505,0.000000,...,0.688998,0.000000,0.053775,0.032631,0.000240,0.041500,0.844000,0.478521,0.041787,0.800000


In [21]:
tracks = X[['index', 'track_id']].merge(audio_features.rename(columns={'id': 'track_id'}), how='left', on='track_id')
tracks

,index,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,0.138,0.388673,0.040086,0.8
1,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,0.138,0.388673,0.040086,0.8
2,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,0.138,0.388673,0.040086,0.8
3,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,0.138,0.388673,0.040086,0.8
4,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,0.138,0.388673,0.040086,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594281,1843050,0kvWwRs634uP7K50NObw2h,0.422111,0.806,0.363636,0.781707,1.0,0.063392,0.090863,0.251,0.1130,0.718,0.563864,0.040903,0.8
14594282,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,0.193,0.428317,0.039684,0.8
14594283,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,0.193,0.428317,0.039684,0.8
14594284,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,0.193,0.428317,0.039684,0.8


In [22]:
tracks = tracks.merge(
    track_info.drop(columns=[
                          'track_name',
                          'track_album_type',
                          'track_album_id',
                          'track_type',
                          'track_album_album',
                          'track_album_disc_number',
                          'track_album_artists',
                          'track_album_name',
                          'track_track'], errors='ignore'),
    how='inner',
    on='track_id'
)

In [23]:
tracks

,index,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,time_signature,track_artists,track_explicit,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,release_date,album_type
0,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
1,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
2,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
3,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
4,1050194,07vS8obfeZbr8H4MgQfXR7,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594281,1843050,0kvWwRs634uP7K50NObw2h,0.422111,0.806,0.363636,0.781707,1.0,0.063392,0.090863,0.251,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.013221,0.0,0.02,0.021442,1.000000,0.996542,album
14594282,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single
14594283,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single
14594284,3755897,3G1RdjwDlaOxavwHfXW39g,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,...,0.8,[{'external_urls': {'spotify': 'https://open.s...,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single


In [24]:
 tracks.drop(columns=['track_artists',
                          'track_name',
                          'track_album_type',
                          'track_album_id',
                          'track_type',
                          'track_album_album',
                          'track_album_disc_number',
                          'track_album_artists',
                          'track_album_name',
                          'track_id',
                          'track_available_markets','track_track'], errors='ignore', inplace=True)

In [25]:
tracks

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,duration_ms,time_signature,track_explicit,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,release_date,album_type
0,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
1,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
2,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
3,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
4,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594281,1843050,0.422111,0.806,0.363636,0.781707,1.0,0.063392,0.090863,0.251,0.1130,...,0.040903,0.8,0.0,0.013221,0.0,0.02,0.021442,1.000000,0.996542,album
14594282,3755897,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,...,0.039684,0.8,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single
14594283,3755897,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,...,0.039684,0.8,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single
14594284,3755897,0.643216,0.385,0.000000,0.805472,1.0,0.027611,0.837349,0.000,0.0568,...,0.039684,0.8,0.0,0.012829,0.0,0.12,0.007797,0.994565,0.999012,single


In [26]:
X

,index,playlist_id,track_id,exists,added_at,added_by_id,album_type,track_explicit,track_album_duration_ms,track_album_explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-07-01 00:00:00+00:00,1,single,0.117647,0.011155,0.117647,...,0.779283,0.823529,0.037813,0.585554,0.020276,0.121894,0.296641,0.389247,0.034479,0.788235
1,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-08-01 00:00:00+00:00,1,single,0.000000,0.013150,0.000000,...,0.726343,0.000000,0.030662,0.855422,0.063395,0.101150,0.428500,0.496725,0.040681,0.700000
2,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-09-01 00:00:00+00:00,1,single,0.250000,0.010305,0.250000,...,0.786116,0.500000,0.061763,0.492219,0.003583,0.216550,0.575500,0.577376,0.031835,0.800000
3,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-10-01 00:00:00+00:00,1,single,0.000000,0.009905,0.000000,...,0.820296,1.000000,0.231644,0.373494,0.000000,0.127000,0.729000,0.399782,0.030590,0.800000
4,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1,2021-11-01 00:00:00+00:00,1,single,0.000000,0.011664,0.000000,...,0.769164,1.000000,0.036867,0.706325,0.078635,0.113450,0.491000,0.414005,0.036061,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594281,1843050,29231,0kvWwRs634uP7K50NObw2h,1,2023-09-01 00:00:00+00:00,1,album,0.000000,0.013528,0.000000,...,0.749529,1.000000,0.065701,0.113589,0.914000,0.475933,0.763000,0.417247,0.041858,0.866667
14594282,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-04-01 00:00:00+00:00,1,single,0.000000,0.016165,0.000000,...,0.795941,0.533333,0.067125,0.055117,0.222907,0.130253,0.741400,0.493158,0.050057,0.793333
14594283,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-05-01 00:00:00+00:00,1,album,0.000000,0.018759,0.000000,...,0.771264,0.333333,0.055533,0.092393,0.142364,0.154644,0.825000,0.476682,0.058123,0.800000
14594284,3755897,32843,3G1RdjwDlaOxavwHfXW39g,0,2020-06-01 00:00:00+00:00,1,album,0.000000,0.013505,0.000000,...,0.688998,0.000000,0.053775,0.032631,0.000240,0.041500,0.844000,0.478521,0.041787,0.800000


In [27]:
columns = tracks.columns.to_list()
columns.remove('index')
columns.remove("album_type")
columns

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'track_explicit',
 'track_album_duration_ms',
 'track_album_explicit',
 'track_album_popularity',
 'track_album_track_number',
 'track_available_markets_len',
 'release_date']

In [28]:
dist = np.sum(X[columns].to_numpy() * tracks[columns].to_numpy(), axis=1) \
        /(np.linalg.norm(X[columns].to_numpy(), axis=1) \
          * np.linalg.norm(tracks[columns].to_numpy(), axis=1))
dist


array([0.92086327, 0.75726587, 0.84006201, ..., 0.82057603, 0.74912472,
       0.89541357])

In [29]:
Z = X[['index', 'exists']]
Z['score'] = dist

/tmp/ipykernel_118/2405601410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Z['score'] = dist


In [30]:
tracks.drop_duplicates(subset=['index'])

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,duration_ms,time_signature,track_explicit,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,release_date,album_type
0,1050194,0.374874,0.123,0.636364,0.688322,1.0,0.042089,0.951807,0.000000,0.0816,...,0.040086,0.8,0.0,0.012958,0.0,0.00,0.001949,0.000000,0.997036,single
18,645550,0.747739,0.453,0.454545,0.737726,1.0,0.035988,0.629518,0.421000,0.0898,...,0.043580,0.8,0.0,0.014082,0.0,0.73,0.003899,1.000000,0.986166,album
19,2630414,0.750754,0.878,0.818182,0.841695,0.0,0.036918,0.079719,0.000531,0.3700,...,0.029494,0.8,0.0,0.009552,0.0,0.00,0.000000,0.000000,0.995059,single
20,3168314,0.607035,0.484,0.636364,0.812692,1.0,0.036815,0.100402,0.000000,0.0865,...,0.039602,0.8,0.0,0.012803,0.0,0.00,0.019493,0.000000,0.988636,album
21,3144464,0.764824,0.447,0.000000,0.753334,1.0,0.063392,0.061847,0.873000,0.0822,...,0.071482,0.8,0.0,0.023055,0.0,0.07,0.003899,1.000000,0.995059,compilation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594243,2211111,0.625126,0.800,0.818182,0.809159,0.0,0.035781,0.000903,0.017400,0.1210,...,0.036901,0.8,0.0,0.011934,0.0,0.05,0.025341,1.000000,1.000000,compilation
14594256,3600703,0.562814,0.888,0.818182,0.773166,1.0,0.057187,0.032530,0.097300,0.9670,...,0.041348,0.8,0.0,0.013364,0.0,0.00,0.001949,0.000000,0.991601,album
14594259,3879581,0.779899,0.667,0.090909,0.844660,0.0,0.444674,0.248996,0.000000,0.0921,...,0.028581,0.8,0.0,0.009259,0.0,0.44,0.003899,1.000000,0.998024,single
14594260,1843050,0.422111,0.806,0.363636,0.781707,1.0,0.063392,0.090863,0.251000,0.1130,...,0.040903,0.8,0.0,0.013221,0.0,0.02,0.021442,1.000000,0.996542,album


In [31]:
Z

,index,exists,score
0,1050194,1,0.920863
1,1050194,1,0.757266
2,1050194,1,0.840062
3,1050194,1,0.818167
4,1050194,1,0.927390
...,...,...,...
14594281,1843050,1,0.939184
14594282,3755897,0,0.851843
14594283,3755897,0,0.820576
14594284,3755897,0,0.749125


In [32]:
Z = Z.groupby('index').agg(
    exists = ('exists', 'max'),
    cosine_max = ('score', 'max'),
    cosine_min = ('score', 'min')
)

In [33]:
Z.reset_index(inplace=True)

In [34]:
Z

,index,exists,cosine_max,cosine_min
0,1,1,1.000000,1.000000
1,3,1,0.860666,0.860666
2,4,1,0.934113,0.934113
3,5,1,0.961346,0.961346
4,6,1,0.971737,0.971737
...,...,...,...,...
2772856,3962029,0,0.823274,0.785212
2772857,3962030,0,0.893901,0.871566
2772858,3962032,0,0.968542,0.963513
2772859,3962034,0,0.820416,0.808404


In [35]:
Z['cosine_diff'] = Z['cosine_max'] - Z['cosine_min']

In [36]:
tracks = tracks.groupby('index').aggregate({'album_type':  lambda x: pd.Series.mode(x)[0]} \
                                           | {column: 'mean' for column in tracks.sample(1).drop(columns=['index', 'album_type']).columns}
                                          ).reset_index()


In [37]:
Z[ tracks.columns] =  tracks
Z

,index,exists,cosine_max,cosine_min,cosine_diff,album_type,danceability,energy,key,loudness,...,tempo,duration_ms,time_signature,track_explicit,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,release_date
0,1,1,1.000000,1.000000,0.000000,single,0.689447,0.635,1.000000,0.803506,...,0.380318,0.053472,0.8,1.0,0.017263,1.0,0.47,0.000000,1.000000,0.997036
1,3,1,0.860666,0.860666,0.000000,album,0.797990,0.766,0.818182,0.861896,...,0.508233,0.035535,0.8,1.0,0.011495,1.0,0.06,0.009747,0.000000,0.993083
2,4,1,0.934113,0.934113,0.000000,compilation,0.556784,0.824,0.454545,0.831126,...,0.676097,0.029587,0.8,0.0,0.009582,0.0,0.76,0.037037,0.994565,0.993083
3,5,1,0.961346,0.961346,0.000000,album,0.620100,0.823,0.818182,0.841680,...,0.390838,0.031455,0.8,1.0,0.010183,1.0,0.56,0.003899,1.000000,0.993577
4,6,1,0.971737,0.971737,0.000000,album,0.885427,0.782,0.818182,0.865537,...,0.568706,0.029018,0.8,1.0,0.009399,1.0,0.68,0.019493,0.994565,0.999012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772856,3962029,0,0.823274,0.785212,0.038063,album,0.503518,0.114,0.090909,0.611435,...,0.468957,0.031799,0.8,0.0,0.010293,0.0,0.08,0.011696,1.000000,0.976285
2772857,3962030,0,0.893901,0.871566,0.022335,compilation,0.251256,0.801,0.090909,0.795579,...,0.605789,0.029549,0.6,0.0,0.009570,0.0,0.13,0.029240,1.000000,0.982213
2772858,3962032,0,0.968542,0.963513,0.005029,album,0.444221,0.312,0.272727,0.698814,...,0.617377,0.038840,0.8,0.0,0.012558,0.0,0.30,0.007797,1.000000,0.975791
2772859,3962034,0,0.820416,0.808404,0.012012,album,0.532663,0.758,0.090909,0.818345,...,0.480270,0.050960,0.8,0.0,0.016456,0.0,0.00,0.007797,0.000000,0.996047


In [38]:
tmp = (Z.drop(columns=['cosine_max', 'cosine_min', 'cosine_diff']) - X.drop(columns=['track_id', 'added_at', 'playlist_id']).select_dtypes(np.number).groupby('index').mean().reset_index())
Z = Z.merge(tmp, suffixes=[None, '_diff'], left_index=True, right_index=True)

In [39]:
Z['user activeness'] = X.groupby('index').aggregate({'track_id': 'count'})

In [40]:
Z['date_diff'] = (X.groupby('index').aggregate({'added_at': 'max'}).reset_index()['added_at'].dt.year / 2024) - tracks['release_date']

In [41]:
Z['album_type'] = Z['album_type'].astype('category')

In [42]:
Z = Z.merge(train[['index', 'playlist_id']], how='left', on='index')

In [43]:
Z = Z.sort_values(by=['playlist_id'])

In [44]:
Z = Z.merge(X.groupby(['playlist_id']).aggregate({'added_by_id': 'max'}).reset_index().rename(columns={'added_by_id': 'users'}), how='left', on='playlist_id')

In [45]:
group = Z['playlist_id']
group

0              0
1              0
2              0
3              0
4              0
           ...  
2772856    36769
2772857    36769
2772858    36769
2772859    36769
2772860    36769
Name: playlist_id, Length: 2772861, dtype: int64

In [46]:
#Z = Z.drop(columns=['playlist_id'])

In [47]:
embeds = pd.read_csv('/kaggle/input/embeds/embdeds.csv')

In [48]:
Z = Z.merge(embeds, how='left', on='playlist_id')

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_val= train_test_split(Z.drop(columns=['index']), test_size=0.2, random_state=42)


In [50]:
X_train = X_train.sort_values(by=['playlist_id'])
X_val = X_val.sort_values(by=['playlist_id'])



In [51]:
X_train, y_train = X_train.drop(columns=['exists']), X_train['exists']
X_val, y_val = X_val.drop(columns=['exists']), X_val['exists']

In [52]:
import catboost

pool_tr = catboost.Pool(X_train.drop(columns=['playlist_id']), y_train, cat_features=['album_type'], group_id=X_train['playlist_id'])
pool_vl = catboost.Pool(X_val.drop(columns=['playlist_id']), y_val, cat_features=['album_type'],  group_id=X_val['playlist_id'])

pars = {
    'n_estimators': 1500,
    'learning_rate': 0.3,
    #'eval_metric': 'F1',
    'loss_function': 'Logloss'
    
}

model = catboost.train(
    pool=pool_tr,
    eval_set=pool_vl,
    params=pars,
    verbose=10,
    early_stopping_rounds=30
)

0:	learn: 0.5324493	test: 0.5325372	best: 0.5325372 (0)	total: 1.96s	remaining: 48m 54s
10:	learn: 0.3169411	test: 0.3166373	best: 0.3166373 (10)	total: 17.9s	remaining: 40m 27s
20:	learn: 0.2830369	test: 0.2825250	best: 0.2825250 (20)	total: 34s	remaining: 39m 55s
30:	learn: 0.2675783	test: 0.2672693	best: 0.2672693 (30)	total: 48.9s	remaining: 38m 38s
40:	learn: 0.2576092	test: 0.2573357	best: 0.2573357 (40)	total: 1m 3s	remaining: 37m 26s
50:	learn: 0.2505370	test: 0.2503238	best: 0.2503238 (50)	total: 1m 16s	remaining: 36m 9s
60:	learn: 0.2445660	test: 0.2443505	best: 0.2443505 (60)	total: 1m 31s	remaining: 35m 53s
70:	learn: 0.2391536	test: 0.2391168	best: 0.2391168 (70)	total: 1m 46s	remaining: 35m 35s
80:	learn: 0.2355787	test: 0.2355986	best: 0.2355986 (80)	total: 2m	remaining: 35m 10s
90:	learn: 0.2324276	test: 0.2325415	best: 0.2325415 (90)	total: 2m 15s	remaining: 34m 58s
100:	learn: 0.2295741	test: 0.2297808	best: 0.2297808 (100)	total: 2m 30s	remaining: 34m 41s
110:	learn:

In [53]:
from sklearn.metrics import f1_score

f1_score(y_val, model.predict(X_val.drop(columns=['playlist_id']), prediction_type='Class'))

0.9260858124302143

In [ ]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_val.drop(columns=['playlist_id']), prediction_type='Probability')[:, 1]
scores = []


for thr in np.linspace(0, 1, 300):
    scores.append((f1_score(y_val, y_pred > thr), thr))

In [ ]:
sorted(scores, key=lambda x: x[0])

In [ ]:
X_val.groupby(['playlist_id']).aggregate({'playlist_id': 'count'})['playlist_id']

In [55]:
import lightgbm as lgb

params = {
    "boosting_type": "gbdt",
    'metric': 'binary_logloss',
    "learning_rate": 0.3,
    "early_stopping_round": 50,
    "reg_lambda": 0.05,
    "num_threads": 4,
    "random_state": 42,
}

model1 = lgb.train(
    params,
    lgb.Dataset(
        X_train.drop(columns=['playlist_id']),
        y_train,
        categorical_feature=['album_type'],
      #  group=X_train.groupby(['playlist_id']).aggregate({'playlist_id': 'count'})['playlist_id']
    ),
    1500,
    [lgb.Dataset(
        X_val.drop(columns=['playlist_id']),
        y_val,
        categorical_feature=['album_type'],
     #   group=X_val.groupby(['playlist_id']).aggregate({'playlist_id': 'count'})['playlist_id']
    )],
    callbacks=[lgb.log_evaluation(10)],
    feature_name=X_train.drop(columns=['playlist_id']).columns.to_list()
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.098499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10776
[LightGBM] [Info] Number of data points in the train set: 2218288, number of used features: 52
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.500806
Training until validation scores don't improve for 50 rounds
[10]	valid_0's binary_logloss: 0.310644
[20]	valid_0's binary_logloss: 0.302077
[30]	valid_0's binary_logloss: 0.2909
[40]	valid_0's binary_logloss: 0.284097
[50]	valid_0's binary_logloss: 0.277697
[60]	valid_0's binary_logloss: 0.273835
[70]	valid_0's binary_logloss: 0.270392
[80]	valid_0's binary_logloss: 0.268955
[90]	valid_0's binary_logloss: 0.266159
[100]	valid_0's binary_logloss: 0.264585
[110]	valid_0's binary_logloss: 0.2

In [75]:
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

cosine = logit(X_train['score'])
playlist_id = X_train['playlist_id']
dart = model1.predict(X_train.drop(columns=['playlist_id']))
cb = model.predict(X_train.drop(columns=['playlist_id']))

In [76]:
X_metrics = pd.DataFrame({
    'playlist_id': playlist_id,
    'cosine': cosine,
    'dart': dart,
    'catboost': cb})
X_metrics

,playlist_id,cosine,dart,catboost
21,0,3.286594,0.956409,2.706870
22,0,1.973347,0.635014,-0.092631
7,0,2.984269,0.362274,-1.035700
2,0,3.025856,1.084681,3.138932
18,0,3.301087,0.926478,2.570394
...,...,...,...,...
2772852,36769,4.548552,1.020264,5.787206
2772848,36769,2.314522,0.219266,-2.148010
2772857,36769,1.519168,0.054523,-4.585602
2772846,36769,2.124756,0.572269,0.864384


In [86]:
X_metrics['cosine'] = X_train['score'].apply(lambda x: 100000 if x > 0.99 else (-100000 if x < 0.01 else logit(x)))

In [89]:
X_metrics.fillna(0, inplace=True)

In [91]:
logreg = LogisticRegression(max_iter=10000).fit(X_metrics, y_train)

In [94]:
cosine = X_val['score'].apply(lambda x: 100000 if x > 0.99 else (-100000 if x < 0.01 else logit(x)))
playlist_id = X_val['playlist_id']
dart = model1.predict(X_val.drop(columns=['playlist_id']))
cb = model.predict(X_val.drop(columns=['playlist_id']))


X_metrics_val = pd.DataFrame({
    'playlist_id': playlist_id,
    'cosine': cosine,
    'dart': dart,
    'catboost': cb})
X_metrics_val.fillna(0, inplace=True)
X_metrics_val

,playlist_id,cosine,dart,catboost
36,0,2.636433,0.989665,2.566523
27,0,2.484528,0.785799,2.072619
26,0,3.448557,0.749371,1.016685
12,0,3.778164,0.549475,0.412782
5,0,2.975241,0.054647,-5.371555
...,...,...,...,...
2772835,36768,3.621923,1.040951,5.926962
2772837,36768,3.496557,0.993792,6.008805
2772830,36768,3.933015,1.051530,6.837947
2772850,36769,4.259475,0.832820,2.008690


In [95]:
f1_score(y_val, logreg.predict(X_metrics_val))

0.9208321381932332

In [ ]:
model.save_model(fname='model')

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(n_estimators=500, learning_rate=5e-2, verbose=50, early_stopping_rounds=50, cat_features=['album_type']).fit(X_train, y_train)

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    n_estimators=1500,
    learning_rate=1e-1,
    verbose=15,
    early_stopping_rounds=50,
    cat_features=['album_type']
).fit(X_train, y_train, eval_set=pool_vl)

In [54]:
sorted(zip(model.feature_importances_, X_train.drop(columns=['playlist_id']).columns))

[(0.0, 'added_by_id'),
 (0.0, 'album_type_diff'),
 (0.0, 'exists_diff'),
 (0.0, 'index_diff'),
 (0.04893575674648858, 'users'),
 (0.07008691500286947, 'track_album_explicit'),
 (0.15609361741578218, 'key_diff'),
 (0.24283380377338393, 'time_signature'),
 (0.257710381972041, 'liveness_diff'),
 (0.28472969621978156, 'liveness'),
 (0.29069080149390325, 'key'),
 (0.33223539138646746, 'track_explicit_diff'),
 (0.46440875521379815, 'track_explicit'),
 (0.47133528096839955, 'mode_diff'),
 (0.5894513615078648, 'track_album_track_number'),
 (0.6048697915212768, 'tempo'),
 (0.6085801942970966, 'tempo_diff'),
 (0.6165331301030768, 'cosine_diff'),
 (0.6188326374458709, 'duration_ms'),
 (0.6218979866738519, 'svd_embed_6'),
 (0.6379394256604083, 'svd_embed_5'),
 (0.689784561102524, 'mode'),
 (0.6983717046916865, 'album_type'),
 (0.7690075485315852, 'user activeness'),
 (0.8104538925985764, 'time_signature_diff'),
 (0.8107592897465217, 'valence'),
 (0.8140137262838536, 'track_album_duration_ms'),
 (0

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_val, clf.predict(X_val))

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_val, clf.predict(X_val))

In [61]:
 model1.predict(X_val.drop(columns=['playlist_id']))

array([ 1.05182905e+00,  7.27609641e-01,  7.64330515e-01, ...,
        1.01888959e+00,  9.08053587e-01, -7.27956447e-05])

In [59]:
from sklearn.metrics import f1_score

f1_score(y_val, model1.predict(X_val.drop(columns=['playlist_id'])))

[LightGBM] [Warning] Unknown parameter: type


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [66]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_val.drop(columns=['playlist_id']), prediction_type='Probability')[:, 1]
scores = []


for thr in np.linspace(0.4, 0.6, 300):
    scores.append((f1_score(y_val, y_pred > thr), thr))

In [67]:
scores

[(0.9254505344378732, 0.4),
 (0.9254636519847895, 0.4006688963210703),
 (0.9254803670031793, 0.4013377926421405),
 (0.9254876666957204, 0.40200668896321073),
 (0.9255088609360426, 0.40267558528428093),
 (0.9255049250132084, 0.4033444816053512),
 (0.9255143092813708, 0.40401337792642145),
 (0.9255326570762812, 0.40468227424749165),
 (0.925528982852757, 0.4053511705685619),
 (0.9255427538608624, 0.4060200668896321),
 (0.9255412705724695, 0.40668896321070236),
 (0.9255408263701284, 0.4073578595317726),
 (0.9255591355434619, 0.4080267558528428),
 (0.9255643205896061, 0.40869565217391307),
 (0.9255476644389975, 0.40936454849498327),
 (0.9255692135773153, 0.4100334448160535),
 (0.9255891520924832, 0.4107023411371238),
 (0.9256483566000597, 0.411371237458194),
 (0.9256822850435293, 0.41204013377926424),
 (0.9257081363618894, 0.41270903010033444),
 (0.9257232513200058, 0.4133779264214047),
 (0.9257395291617737, 0.41404682274247495),
 (0.9257400306882486, 0.41471571906354515),
 (0.9257670009141

In [108]:
import gc
gc.collect()

del Z_t

In [56]:
test = pd.read_csv('/kaggle/input/cmc-ml-spotify-recommendations/test.csv')

In [113]:
X = test.merge(df, how='left', on='playlist_id')
tracks = X[['index', 'track_id']].merge(audio_features.rename(columns={'id': 'track_id'}), how='left', on='track_id')
tracks = tracks.merge(
    track_info.drop(columns=[
                          'track_name',
                          'track_album_type',
                          'track_album_id',
                          'track_type',
                          'track_album_album',
                          'track_album_disc_number',
                          'track_album_artists',
                          'track_album_name',
                          'track_track'], errors='ignore'),
    how='inner',
    on='track_id'
)
tracks.drop(columns=['track_artists',
                      'track_name',
                      'track_album_type',
                      'track_album_id',
                      'track_type',
                      'track_album_album',
                      'track_album_disc_number',
                      'track_album_artists',
                      'track_album_name',
                      'track_id',
                      'track_available_markets','track_track'], errors='ignore', inplace=True)

dist = np.sum(X[columns].to_numpy() * tracks[columns].to_numpy(), axis=1) \
        /(np.linalg.norm(X[columns].to_numpy(), axis=1) \
          * np.linalg.norm(tracks[columns].to_numpy(), axis=1))
dist


array([0.96251135, 0.88751366, 0.91867268, ..., 0.70921637, 0.72162643,
       0.68915242])

In [115]:
#new
Z_t = X[['index']]
Z_t['score'] = dist
Z_t = Z_t.groupby('index').agg(
    cosine_max = ('score', 'max'),
    cosine_min = ('score', 'min')
).reset_index()

/tmp/ipykernel_118/3341801893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Z_t['score'] = dist


In [116]:
Z_t.isna().sum(axis=0)

index           0
cosine_max    863
cosine_min    863
dtype: int64

In [117]:


tracks = tracks.groupby('index').aggregate({'album_type':  lambda x: pd.Series.mode(x)[0]} \
                                           | {column: 'mean' for column in tracks.sample(1).drop(columns=['index', 'album_type']).columns}
                                          ).reset_index()
Z_t[ tracks.columns] = tracks

tmp = (Z_t.drop(columns=['cosine_max', 'cosine_min']) - X.drop(columns=['track_id', 'added_at', 'playlist_id']).select_dtypes(np.number).groupby('index').mean().reset_index())
Z_t = Z_t.merge(tmp, suffixes=[None, '_diff'], left_index=True, right_index=True)
Z_t['user activeness'] = X.groupby('index').aggregate({'track_id': 'count'})
Z_t['date_diff'] = (X.groupby('index').aggregate({'added_at': 'max'}).reset_index()['added_at'].dt.year / 2024) - tracks['release_date']
Z_t['album_type'] = Z_t['album_type'].astype('category')
Z_t = Z_t.merge(train[['index', 'playlist_id']], how='left', on='index')
Z_t = Z_t.sort_values(by=['playlist_id'])
#Z_t = Z_t.drop(columns=['playlist_id'])

In [118]:
Z_t.isna().sum(axis=0)

index                                     0
cosine_max                              863
cosine_min                              863
album_type                                0
danceability                            514
energy                                  514
key                                     514
loudness                                514
mode                                    514
speechiness                             514
acousticness                            514
instrumentalness                        514
liveness                                514
valence                                 514
tempo                                   514
duration_ms                             514
time_signature                          514
track_explicit                            0
track_album_duration_ms                   0
track_album_explicit                      0
track_album_popularity                    0
track_album_track_number                  0
track_available_markets_len     

In [112]:
tmp

,acousticness,added_by_id,album_type,danceability,duration_ms,energy,index,instrumentalness,key,liveness,...,speechiness,tempo,time_signature,track_album_duration_ms,track_album_explicit,track_album_popularity,track_album_track_number,track_available_markets_len,track_explicit,valence
0,-0.074198,NaN,NaN,0.036948,0.006711,0.039123,0,-0.303748,-0.426573,-0.144814,...,-0.029088,-0.097397,0.006154,0.002158,-0.107692,-0.010923,0.001290,0.031355,-0.107692,0.186222
1,-0.140467,NaN,NaN,-0.001148,0.015136,0.217744,0,-0.018738,-0.383163,0.047616,...,-0.019161,0.000598,0.000775,0.004868,-0.011628,-0.185055,0.007010,-0.925563,-0.011628,0.066793
2,0.215293,NaN,NaN,-0.029242,-0.006434,0.022743,0,0.105433,0.448658,0.235316,...,-0.004250,0.033723,0.064825,-0.002069,0.000000,-0.180151,-0.011667,-0.482258,0.000000,-0.091883
3,0.850815,NaN,NaN,-0.179330,0.166906,-0.478678,0,0.862960,-0.180556,-0.127955,...,-0.048852,0.036962,-0.200000,0.053675,0.000000,-0.684528,-0.013358,0.035039,0.000000,-0.263039
4,-0.073475,NaN,NaN,-0.033090,0.002591,0.079803,0,0.240422,-0.341598,-0.085562,...,-0.046949,0.029679,0.003030,0.000833,-0.030303,-0.030000,-0.000561,0.031291,-0.030303,-0.175179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188451,-0.503026,NaN,NaN,0.003736,0.019481,0.493977,0,-0.707281,-0.354649,-0.070418,...,-0.051466,0.070194,0.000980,0.006265,0.000000,0.245759,0.000603,0.187255,0.000000,-0.030707
1188452,0.437852,NaN,NaN,-0.027808,0.015747,0.089066,0,0.219385,0.094848,-0.053715,...,-0.031387,-0.016798,0.013333,0.005064,-0.004444,0.237856,0.001098,0.521624,-0.004444,0.144374
1188453,-0.178787,NaN,NaN,-0.112753,-0.004823,0.040260,0,-0.043063,-0.366053,0.100641,...,0.013337,-0.046257,0.000000,-0.001551,0.000000,0.181135,0.008045,0.080105,0.000000,-0.402290
1188454,0.054036,NaN,NaN,-0.056740,-0.008642,-0.462913,0,-0.267991,-0.347826,0.161693,...,-0.113617,-0.169324,0.013043,-0.002779,0.804348,0.195652,0.006695,0.394140,0.804348,0.059165


In [60]:
Z_t['exists_diff'] = [0.0] * Z_t.shape[0]

In [61]:
Z_t = Z_t.merge(X.groupby(['playlist_id']).aggregate({'added_by_id': 'max'}).reset_index().rename(columns={'added_by_id': 'users'}), how='left', on='playlist_id')

In [62]:
Z_t = Z_t.merge(embeds, how='left', on='playlist_id')

In [93]:
Z_t['cosine_diff'] = Z_t['cosine_max'] - Z_t['cosine_min']

In [63]:
 set(Z_t.columns) - set(X_train.columns)

{'index'}

In [78]:
model.predict(X_train.sample(50).drop(columns=['playlist_id']))

array([  3.54549285,  -2.99019277,   3.40645769,  -3.52708541,
         0.61853402,  -0.6072599 ,  -3.58847854,  -5.58430415,
        -8.32970278,   0.4354576 ,  -5.01495366,   4.45763546,
       -10.02083604,  -8.73086481,   0.7809083 ,   1.77039251,
        -5.5669463 ,   4.16066295,  -4.5252325 ,   1.78604354,
         3.67046431,   2.96388575,  -0.42130622,   4.2399485 ,
         4.05305386,  -6.86413225,  -4.39154181,   2.06437032,
        -7.64443235,  -8.10850182,  -3.56141374,   3.15676383,
        -5.95065792,   2.53931827,  -7.75434537,   3.39639124,
        -7.863046  ,  -5.29263486,  -4.73077596,  -9.28912894,
        -9.31231438,  -3.64298837, -10.39506181, -13.03891517,
        -5.18965105,   8.4843165 , -19.70749949,   2.94998013,
         5.14863942,   1.61409283])

In [86]:
Z_t['album_type'] = Z_t['album_type'].astype(X_train['album_type'].dtype)

In [90]:
X_train['album_type'].dtype == Z_t['album_type'].dtype

True

In [99]:
pd.DataFrame({
    "index": Z_t['index'], 
    "exists": model.predict(Z_t[X_train.drop(columns=['playlist_id']).columns], prediction_type='Probability', ntree_end=440)[:, 1] > 0.48}).to_csv("/kaggle/working/submission1.csv",index=False )

In [ ]:
set(X_train.columns.to_list()) - set(Z_t.columns.to_list()) 

In [ ]:
X_train['exists_diff'].value_counts()

In [ ]:
pd.read_csv('/kaggle/working/submission.csv')

In [ ]:
pd.read_csv('/kaggle/working/submission.csv')